# Import Modules

In [4]:
import dataset as ds
import config as cfg
from evaluate import Evaluate
import tools


from sklearn.model_selection import train_test_split
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Rescaling, Dropout
from tensorflow.keras.metrics import Precision, Recall

from clearml import Task
import clearml
clearml.browser_login()


# ClearML
https://app.clear.ml/dashboard

# Import Dataset

In [22]:
# Base paths for the images and labels
train_images_path = f'{cfg.BASE_DATASET}/images/train'
train_labels_path = f'{cfg.BASE_DATASET}/labels/train'
val_images_path = f'{cfg.BASE_DATASET}/images/valid'
val_labels_path = f'{cfg.BASE_DATASET}/labels/valid'

# Base path for metadata
metadata_path = f'{cfg.BASE_DATASET}/metadata.json'

# Create the DataFrames for the train and validation sets
train_df = ds.create_dataframe(train_images_path, train_labels_path, metadata_path)
valid_df = ds.create_dataframe(val_images_path, val_labels_path, metadata_path)

# Test Parameters

In [23]:
project_name= cfg.CLF_PROJECT_NAME + '/epoch_test'
dataset_name = 'test250'
dataset_dir = cfg.CLF_DATASET_DIR + f'/{dataset_name}'
project_dir = f'{cfg.CLF_PROJECT_DIR}/{dataset_name}/'
class_names = cfg.CLF_CLASS_NAMES

zoom_factor = 1.5

epochs = 50
N = 20

RUN = 5








task_name = 'epoch_test'
task_name = f'{task_name}-{epochs}-{RUN}'

#augmentation_metadata = {
#    'methods': {        
#        'flip': {
#            'parameters': {
#                'orientation': 'h',  # Could be 'h' for horizontal or 'v' for vertical
#                'p': 1.0  # Probability of applying the augmentation
#            },
#            'apply_to_percentage': 0.5  # 50% of the training images
#        }        
#    }
#}

augmentation_metadata = {
    'methods': {        
        'bnc': {
            'parameters': {
                'alpha': 1.0,
                'beta': 60
            },
            'apply_to_percentage': 0.5  # 50% of the training images
        }        
    }
}

train_size = 2500
val_size = int(train_size * 0.2)


# Create new datasets

In [25]:
seed_time = tools.generate_seed()

# fitler by weather

# filter by location 
#filtered_train_df = train_df[(train_df['location'] == 'Boston')]
#filtered_valid_df = valid_df[(train_df['location'] == 'Boston')]
#
train_df = train_df[(train_df['location'] == 'Palo Alto')]
valid_df = valid_df[(valid_df['location'] == 'Palo Alto')]

filtered_train_df = train_df[(train_df['weather'] == 2) | (train_df['weather'] == 3)]
filtered_valid_df = valid_df[(valid_df['weather'] == 2) | (valid_df['weather'] == 3)]

#
#filtered_train_df = train_df[(train_df['location'] == 'Reno Tahoe')]
#filtered_valid_df = valid_df[(train_df['location'] == 'Reno Tahoe')]
#
#filtered_train_df = train_df[(train_df['location'] == 'Osh Kosh')]
#filtered_valid_df = valid_df[(train_df['location'] == 'Osh Kosh')]

# Create a combined stratification key
filtered_train_df['stratify_key'] = filtered_train_df['ac'] + '_' + filtered_train_df['weather'].astype(str)
filtered_valid_df['stratify_key'] = filtered_valid_df['ac'] + '_' + filtered_valid_df['weather'].astype(str)

# Now use this stratification key in train_test_split
_, test_train_df = train_test_split(
    filtered_train_df,
    test_size=train_size,  # Number of items you want in your sample
    stratify=filtered_train_df['stratify_key'],  # Stratify based on the combined column
    random_state=seed_time  # Ensures reproducibility
)

_, test_val_df = train_test_split(
    filtered_valid_df,
    test_size=val_size,  # Number of items you want in your sample
    stratify=filtered_valid_df['stratify_key'],  # Stratify based on the combined column
    random_state=seed_time  # Ensures reproducibility
)




C:\Users\tomev\AppData\Local\Temp\ipykernel_46176\1108319857.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train_df['stratify_key'] = filtered_train_df['ac'] + '_' + filtered_train_df['weather'].astype(str)
C:\Users\tomev\AppData\Local\Temp\ipykernel_46176\1108319857.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_valid_df['stratify_key'] = filtered_valid_df['ac'] + '_' + filtered_valid_df['weather'].astype(str)


In [ ]:
# create dataset directory from dataframes above
ds.create_sub_dataset(dataset_dir, test_train_df, test_val_df, class_names)


# correct single class labels to accomodate for multi-class classification
ds.correct_dataset_labels(dataset_dir, test_train_df, test_val_df, class_names)

# augment dataset
ds.augment_dataset(dataset_dir, augmentation_metadata)

In [ ]:
# Pre-processing to AID classification (apply zoom factor to all images)
#ds.pre_process_dataset_for_classification(dataset_dir, zoom_factor)

# create class folders within train and valid directories for keras format
ds.reorganize_dataset_for_keras(dataset_dir)

# Image Classifier

In [ ]:
tf.keras.utils.set_random_seed(42)

In [ ]:

train_dir = os.path.join(dataset_dir,'images','train')
train_aug_dir = os.path.join(dataset_dir,'images','train-aug')
valid_dir = os.path.join(dataset_dir,'images','valid')

batch_size = 32 
img_height = 256
img_width = 256

# load datasets using keras
train_data = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    seed=42,
    image_size=(img_height, img_width),
    shuffle=True)

train_aug_data = tf.keras.utils.image_dataset_from_directory(
    train_aug_dir,
    batch_size=batch_size,
    seed=42,
    image_size=(img_height, img_width),
    shuffle=True)

valid_data = tf.keras.utils.image_dataset_from_directory(
    valid_dir,
    batch_size=batch_size,
    seed=42,
    image_size=(img_height, img_width),    
    shuffle=True)


print(class_names)

In [ ]:
# hyper-parameters
hyper_params = {
    'dataset': dataset_name,
    'original_dataset_size': {'train_size': train_size, 'val_size': val_size},
    'train_size': tools.count_images(train_dir),
    'train_aug_size': tools.count_images(train_aug_dir),
    'valid_size': tools.count_images(valid_dir),
    'epochs': epochs, 
    'N': N,
    'zoom_factor': zoom_factor, 
    'batch_size': batch_size, 
    'img_height': img_height, 
    'img_width': img_width, 
    'class_names': class_names
    }

In [ ]:
# test visualise 
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_aug_data.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
# convert labels to one-hot encoding to allow for more metrics to be tracked
def one_hot_enc(image, label):
    return image, tf.one_hot(label, len(class_names))

train_data = train_data.map(one_hot_enc)
train_aug_data = train_aug_data.map(one_hot_enc)
valid_data = valid_data.map(one_hot_enc)

# check one hot new shape
for image_batch, labels_batch in train_data:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
# Set up data pipelines for train and val: 
# cache datasets to reduce I/O operations 
# shuffle training data for robustness
# AUTOTUNE to dynamically optimize prefetching

AUTOTUNE = tf.data.AUTOTUNE
train_data = train_data.cache().shuffle(1000, seed=42).prefetch(buffer_size=AUTOTUNE)
train_aug_data = train_aug_data.cache().shuffle(1000, seed=42).prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)




In [ ]:
# get list of validation labels (y_true) for evaluation
validation_labels = []
for images, labels in valid_data:
    validation_labels.append(labels.numpy())

validation_labels = np.concatenate(validation_labels)

In [ ]:
# CNN for image classification
def get_model(): 
  # number of classes
  num_classes = len(class_names) 
  model = Sequential([
    Input(shape=(img_height, img_width, 3)), # inputs shape, height, width, channels (RGB)
    Rescaling(1./255), # normalize pixel values
    Conv2D(16, 3, padding='same', activation='relu'), # 16 filters, 3x3 kernel, relu activation
    MaxPooling2D(), # performs 2D max pooling
    Conv2D(32, 3, padding='same', activation='relu'), # 32 filters, 3x3 kernel, relu activation
    MaxPooling2D(), # 2D max pooling
    Conv2D(64, 3, padding='same', activation='relu'), # 64 filters, 3x3 kernel, relu activation
    MaxPooling2D(), # 2D max pooling
    Dropout(0.2), # dropout layer to increase regularization and reduce overfitting
    Flatten(), # flattens output of previos layer to 1D
    Dense(128, activation='relu'),      
    Dense(num_classes, activation='softmax') 
  ])
  
  # compile model  
  model.compile(optimizer='adam', # use Adam optimizer
                loss='categorical_crossentropy', # use categorical crossentropy loss for mutliclass classification
                metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]) # trach accuracy, precision, recall (and loss)

  # print model summary
  #model.summary()
  
  return model

In [ ]:
# Train Pure

# local logs directory
logs_dir=cfg.CLF_PROJECT_DIR
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logs_dir)

#connect to clearml
task = Task.init(project_name=project_name, task_name=f"{task_name}-pure")
logger = task.get_logger()

# clearml hyperparameters
task.connect(hyper_params)

# ensure model from scratch and get model

model = get_model()

# train model
print('Training model...')
train_hst = model.fit(
    train_data, 
    epochs=hyper_params['epochs'], 
    validation_data=valid_data,
    callbacks=[tensorboard_callback]
    )

# predict on validation set
print('Predicting on validation set...')
y_pred = model.predict(valid_data)

# evaluate
pure_eval = Evaluate(train_hst, validation_labels, y_pred, class_names, aug=False, sf=3)

# send metrics to clearML
pure_eval.log_metrics(task, logger, N, None, hyper_params)


# close task
print("done")


In [ ]:
# Train Augmented

# local logs directory, callbacks allow for real time metrics in clearML
logs_dir=cfg.CLF_PROJECT_DIR
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logs_dir)

#connect to clearml
task = Task.init(project_name=project_name, task_name=f"{task_name}-aug")
logger = task.get_logger()


# clearml hyperparameters
task.connect(hyper_params)

# ensure model from scratch and get model
model_aug = get_model()

# train model
print('Training model...')
train_aug_hst = model_aug.fit(
    train_aug_data, 
    epochs=hyper_params['epochs'], 
    validation_data=valid_data, 
    callbacks=[tensorboard_callback]
    )

# predict on validation set
print('Predicting on validation set...')
y_pred_aug = model_aug.predict(valid_data)

# evaluate
aug_eval = Evaluate(train_aug_hst, validation_labels, y_pred_aug, class_names, aug=True, sf=3)

# send metrics to clearML
aug_eval.log_metrics(task, logger, N, augmentation_metadata, hyper_params) 
 
# close task
print("done")


In [ ]:
#task.close()

# Evaluation

All metrics have been fully logged in ClearML, this section is just for quick performance metrics not complete evaluation

In [ ]:
# pure accuracy

#pure_eval.average_accuracy(True)
#print()
#pure_eval.std_accuracy(True)
#print()
pure_eval.best_accuracy(True)
print()
#pure_eval.last_accuracy(True)
#print()



In [ ]:
# augmented accuracy

#aug_eval.average_accuracy(True)
#print()
#aug_eval.std_accuracy(True)
#print()
aug_eval.best_accuracy(True)
print()
#aug_eval.last_accuracy(True)
#print()


In [ ]:
# Area under the ROC curve 
pure_eval.overall_auc(True)
aug_eval.overall_auc(True)
print()

In [ ]:
# Max validation F1 score 
pure_eval.max_f1(True)
aug_eval.max_f1(True)
print()

In [ ]:
# loss

# minimum vakidation loss
#pure_eval.min_loss(True)
#aug_eval.min_loss(True)
#print()

# difference in average loss between training and validation sets for the last N epochs
pure_eval.diff_avg_loss_lastN(N,True)
aug_eval.diff_avg_loss_lastN(N,True)
print()

# standard deviation of loss for the last N epochs
#pure_eval.std_loss_lastN(N,True)
#aug_eval.std_loss_lastN(N,True)
#print()




# Class Specific Evaluation

In [ ]:
# pure class auc

#pure_eval.confusion_matrix_class_report(False)
#print()

pure_eval.class_auc(True)
print()


In [ ]:
# augmented class auc
#aug_eval.confusion_matrix_class_report(True)
#print()

aug_eval.class_auc(True)    
print()

